# INTRODUCING HELEN

**FRieDGE, a smart fridge than can talk and hear. This user-friendly fridge leverages NLP, Voice Recognition and Computer vision to simplify user's cooking experience and allows them to gain moe autonomy. The interactions between the fridge and the users are enabled by "Helen", a voice-based AI-powered digital assistant**

Helen requires a wide variety of functions to work, each of these functions are triggered by certain key words. Each function leverages a certain part of Natural Language Processing in order to carryout it's task

We begin by introducing certain libraries and defining these functions

**Importing libraries**

In [1]:
import spacy
import time
import datetime
import winsound
import requests
import speech_recognition as sr
import pyaudio
from gtts import gTTS
import os
import pywhatkit
import pyautogui
import urllib.request
from urllib.request import urlopen
import webbrowser
import re
import wikipedia
from random import randint
import pandas as pd
import numpy as np
from spacy import displacy
nlp = spacy.load("en_core_web_lg")

**Sentiment Intensity Analyzer** 

Sentiment Intensity Analyser from NLTK enables us to determine the general sentiment of a given sentence by calculating it's polarity score and observing it's compound score

In [2]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

**Countdown**

in order to put a timer on lines of codes

In [3]:
import time
  
# define the countdown func.
def countdown(t):
    
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:02d}:{:02d}'.format(mins, secs)
        print(timer, end="\r")
        time.sleep(1)
        t -= 1
      
    #print('Fire in the hole!!')

**Get weather api**

In order to get real time weather updates of any city in the world we use the get weather api

In [4]:
api_key = "0e9b7ea5779519b07e0dc07ebfb0d0e0"

def get_weather(city_name):
    api_url = "http://api.openweathermap.org/data/2.5/weather?q={}&appid={}".format(city_name, api_key)

    response = requests.get(api_url)
    response_dict = response.json()

    response_dict
    weather = response_dict["weather"][0]["description"]
    temperature = str(round(response_dict['main']['temp'] - 273.15)) +' '+ 'celcius'
    min_temp = str(round(response_dict['main']['temp_min'] - 273.15)) +' '+ 'celcius'
    max_temp = str(round(response_dict['main']['temp_max'] - 273.15)) +' '+ 'celcius'

    if response.status_code == 200:
        return str(weather) +" "+'with temperature'+' '+str(temperature) + '......... ' + 'the minimum temperature for the day will be ' + str(min_temp) + ' ' + 'with maximum temperature ' + str(max_temp)
    else:
        print('[!] HTTP {0} calling [{1}]'.format(response.status_code, api_url))
        return None

get_weather('Madrid')

'clear sky with temperature 20 celcius......... the minimum temperature for the day will be 19 celcius with maximum temperature 20 celcius'

**Chatbot function**

A function to identify and extract entites labelled "GPE" in order to determine weather at given city

In [5]:
def chatbot(statement):
    
    #weather = nlp("Current weather in a city")
    statement = nlp(statement)
    print(statement)
    #min_similarity = 0.0

#if weather.similarity(statement) >= min_similarity:
    for ent in statement.ents:
        if ent.label_ == "GPE": # GeoPolitical Entity
            city = ent.text
            city_weather = get_weather(city)

            if city_weather is not None:
                return "In " + city + ", the current weather is: " + city_weather
            else:
                return "Something went wrong."

            break
        else:
            print('NO GPE FOUND')
            pass
    
    print('WEATHER BOT')
    #print(city_weather)
    print('----------')

#chatbot('play middle child')

**Redefining chatbot function**


In [6]:
def chatbot2(statement):
            #weather = nlp("Current weather in a city")
            statement = nlp(statement)
            #min_similarity = 0.0

            #if weather.similarity(statement) >= min_similarity:
            for ent in statement.ents:
                if ent.label_ == "GPE": # GeoPolitical Entity
                    city = ent.text
                    break
                else:
                    return "You need to tell me a city to check."
            city_weather = get_weather(city)
            if (city_weather is not None) & (len(doc_token) < 1):
                return "In " + city + ", the current weather is: " + city_weather

            elif (city_weather is not None) & (len(doc_token) >= 1) & (len(adj) >= 1) & (similarity_doc_adj < 0.55):
                return "No.... "+"in " + city + ", the current weather is: " + city_weather

            elif (city_weather is not None) & (len(doc_token) >= 1) & (len(adj) >= 1) & (similarity_doc_adj > 0.55):
                return "Yes.... "+"in " + city + ", the current weather is: " + city_weather
            else:
                return "Something went wrong."
        #else:
                #return "Sorry I don't understand that. Please rephrase your statement."


**Speech to text function to handle errors**

As most chatbots such as Siri and Alexa, we leverage Speech Recogniation in order to convert audio to text for further analysis 

In [7]:
def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.

    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.record(source,duration=4)
        
    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response

**Youtube function**

In order to let the user play their music

In [8]:
def youtube(x):
    doc2 = x.lower()
    doc2 = x.replace('play','')
    search_keyword=doc2.replace(' ','')
    html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + search_keyword)
    video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
    print("https://www.youtube.com/watch?v=" + video_ids[0])
    webbrowser.open(f"https://www.youtube.com/watch?v={video_ids[0]}")

**Adding new entity's to pipeline**

Introducing kitchen ingredients as 'PRODUCT' entities so that they may be identified and extracted

In [9]:
for ent in nlp('chicken').ents:
    print(ent,ent.label_)

In [10]:
from spacy.pipeline import EntityRuler

In [11]:
df_ingrident = pd.read_excel('ingredient.xlsx')

In [12]:
df_ingrident['Ingredient'] = df_ingrident['Ingredient'].apply(lambda x: x.lower())

In [13]:
ingrident_list = list(df_ingrident['Ingredient'])

In [14]:
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "PRODUCT", "pattern": str(i)} for i in ingrident_list]
ruler.add_patterns(patterns)

doc = nlp("We can cook with chicken.")
print([(ent.text, ent.label_,spacy.explain(ent.label_)) for ent in doc.ents])

[('chicken', 'PRODUCT', 'Objects, vehicles, foods, etc. (not services)')]


In [15]:
for ent in nlp('chia seed').ents:
    print(ent)

chia seed


**Phone book dataframe for whatsapp**

In [3]:
df_phone_code = pd.read_excel('codes.xlsx',sheet_name='Phone Codes')
phone_codes = dict(zip(df_phone_code['Person'],df_phone_code['Phone Code']))
phone_codes['Eleanor'] = '+34627166596'
phone_codes['Jose Mario'] = '+34681226022'
phone_codes

{'Eleanor': '+34627166596', 'Jose Mario': '+34681226022'}

**Whatsapp function**

Identifying "PERSON" entity to whom the user wants to send a message

In [16]:
def whatsapp(text):
    
    person = []
    for ent in nlp(text).ents:
        if ent.label_ == 'PERSON':
            person.append(ent)
            print(ent,ent.label_)
    
    mytext = "what message would you like to send?"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("whatsapp.mp3")
    os.system("whatsapp.mp3")
    countdown(4)
    winsound.Beep(500, 200)
    
    init_rec = sr.Recognizer()
    for i in range(100):
            
            guess = recognize_speech_from_mic(init_rec, sr.Microphone())
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
    
   
    mytext = "sending"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("whatsapp2.mp3")
    os.system("whatsapp2.mp3")
    countdown(2)
  
    hour = datetime.datetime.now().strftime('%H')

    phone_no = str(phone_codes[str(person[0])])

    message = guess["transcription"]

    time_hour = int(hour)

    minute = str(int(datetime.datetime.now().strftime('%M')) + 1)
    time_min = int(minute)

    pywhatkit.sendwhatmsg(phone_no=phone_no,message=message,time_hour=time_hour,time_min=time_min)
    
    mytext = "your message has been sent"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)

    
    myobj.save("whatsapp3.mp3")
    os.system("whatsapp3.mp3")
    countdown(4)

**Ingredient dataframe for recipes**

To indentify ingredients about to expire

In [17]:
df_ingredient_exp = {'ingridents':['chicken','coconut','milk'],'exp_date':['2021-05-10','2021-05-11','2021-05-09']}
df_ingredient_exp = pd.DataFrame(df_ingredient_exp)
df_ingredient_exp['exp_date'] = df_ingredient_exp['exp_date'].astype('datetime64')
df_ingredient_exp['day_left'] = df_ingredient_exp['exp_date'] - datetime.datetime.today() 
df_ingredient_exp['day_left'] = (df_ingredient_exp['day_left'] / np.timedelta64(1,'D')).astype(int) 
df_ingredient_exp['day_left'] = df_ingredient_exp['day_left'].astype(str)
df_ingredient_exp['text'] = df_ingredient_exp['ingridents'] +' in '+df_ingredient_exp['day_left']+" "+'days'
df_ingredient_exp    

,ingridents,exp_date,day_left,text
0,chicken,2021-05-10,0,chicken in 0 days
1,coconut,2021-05-11,1,coconut in 1 days
2,milk,2021-05-09,0,milk in 0 days


**Recipe function**

Once ingredients have been identified, a corresponsing recepie is given

In [19]:
def recipe_function():
    
    mytext = "The following ingredients will expire in the coming days" + str(list(df_ingredient_exp['text']))
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("recipe_step1.mp3")
    os.system("recipe_step1.mp3")
    countdown(round(len(mytext)*0.1))

           
    mytext = "what would you like to cook with?"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("recipe_step2.mp3")
    os.system("recipe_step2.mp3")
    countdown(round(len(mytext)*0.095))
    winsound.Beep(500, 200)
    
    #ok lets cook with
    init_rec = sr.Recognizer() 
    for i in range(100):
        print('Speak!')
        guess = recognize_speech_from_mic(init_rec, sr.Microphone())
        print(guess['transcription'])
        if guess["transcription"] and any(x in guess['transcription'].lower() for x in list(df_ingrident['Ingredient'])):
            break
            
    for ent in nlp(guess['transcription']).ents:
        df_recipe = pd.read_excel('recipes.xlsx',sheet_name=f"{ent}")
    
    #df_recipe['Instructions'].iloc[0] - recipe name
    mytext = "Would you like to cook " + df_recipe['Instructions'].iloc[0]
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("recipe_step3.mp3")
    os.system("recipe_step3.mp3")
    countdown(round(len(mytext)*0.095))
    winsound.Beep(500, 200)
    
    #yes or no but in our case a definte yes
    init_rec = sr.Recognizer() 
    for i in range(100):
        print('Speak!')
        guess = recognize_speech_from_mic(init_rec, sr.Microphone())
        print(guess['transcription'])
        if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['yes','do','it']):
            break
    
    for num in range(1,len(df_recipe['Instructions'])):
        recepie_loop = True
        while recepie_loop:
            mytext = df_recipe['Instructions'].iloc[num]
            language = 'en'
            myobj = gTTS(text=mytext, lang=language, slow=False)
            myobj.save(f"step{num}.mp3")
            os.system(f"step{num}.mp3")
            countdown(round(len(mytext)*0.095))
            winsound.Beep(500, 200)

            if 'finally' in mytext:
                recepie_loop = False
                break

            else:        
                init_rec = sr.Recognizer() 
                for i in range(100):


                    print('Speak!')
                    guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                    print(guess['transcription'])
                    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['repeat','again']):
                        os.system("repeat.mp3")
                        countdown(1)
                        break

                    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['next','ok','okay']):
                        break

            if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['next','ok','okay']):
                recepie_loop=False
                os.system("next.mp3")
                #countdown(1)
                break

**Joke function**

Using NLTK, we determine the sentiment of a gixen text

In [20]:
def joke(text):
    
    if sia.polarity_scores(text)['compound'] > 0.1:

        mytext = 'Call me crazy but I love to see other people happy, It makes me happy'
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save("happy.mp3")
        os.system("happy.mp3")
        countdown(round(len(mytext)*0.095))

    if sia.polarity_scores(text)['compound'] < 0.1:

        mytext = "Well we can't have you feeling that way the whole day, do you want to hear a joke?"
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save("joke.mp3")
        os.system("joke.mp3")
        countdown(7)
        winsound.Beep(500, 200)

        init_rec = sr.Recognizer() 
        for i in range(100):
            print('Speak!')
            guess = recognize_speech_from_mic(init_rec, sr.Microphone())
            print(guess['transcription'])
            if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['yes','no','ok','sure','tell','okay']):
                break
            if not guess["success"]:
                break

        if any(x in guess['transcription'].lower() for x in ['yes','please','sure','tell','okay']):

            joke = True
            while joke:


                df_joke = pd.read_excel('jokes v1.xlsx')
                mytext = df_joke['Jokes'][randint(0,len(df_joke['Jokes']))]
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("joke_text.mp3")
                os.system("joke_text.mp3")
                countdown(round(len(mytext)*0.095))

                mytext = "lol,would you like to hear another one?"
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("joke_again.mp3")
                os.system("joke_again.mp3")
                countdown(round(len(mytext)*0.095))
                winsound.Beep(500, 200)

                init_rec = sr.Recognizer() 
                for i in range(100):

                    print('Speak!')
                    guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                    print(guess['transcription'])

                    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['yes','ok','sure','tell','okay','another']):
                        break
                    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['no','thank you']):
                        break

                if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['no','thank you']):
                        joke = False

**Adjusting volume**

In [21]:
def volume_adjust(x):
    
    if any(y in x.lower() for y in ['up','increase']):
        pyautogui.press('volumeup',presses=20)
    
    if any(y in x.lower() for y in ['down','lower']):
        pyautogui.press('volumedown',presses=15)
        
    
    if any(y in x.lower() for y in ['maximum','max']):
        pyautogui.press('volumeup',presses=100)
        
    
    if any(y in x.lower() for y in ['minimum','mute']):
        pyautogui.press('volumemute')    

**Describe function**

Gives a brief wikipedia summary when "PRODUCT" entity is identified

In [22]:
def describe(x):
    for ent in nlp(x).ents:
        mytext = wikipedia.summary(str(ent),2)
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save("describe.mp3")
        os.system("describe.mp3")
        countdown(round(len(mytext)*0.095))

# HI HELEN TO START

In [53]:
repeat = True
while repeat:
    
    init_rec = sr.Recognizer()
    for i in range(1000):

        print('Speak!')
        guess = recognize_speech_from_mic(init_rec, sr.Microphone())
        print(guess['transcription'])
        if (guess['transcription'] != None) and ('helen' in guess['transcription'].lower())==True:
            break
        if not guess["success"]:
            break
        print(f"{i}: NEXT LOOP")


    # 1
    mytext = 'Hi.. how can I help you'
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("welcome.mp3")
    os.system("welcome.mp3")
    countdown(4)
    winsound.Beep(500, 200)

    # 2
    for i in range(100):
                print('Speak!')
                guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                print(guess['transcription'])
                if guess["transcription"]:
                    break
                if not guess["success"]:
                    break

    loop = True
    while loop:

        try:


           # 3                
            doc = nlp(guess['transcription'])
            doc_token = []

            for token in doc:
                if token.pos_ in ['ADJ','VERB','NN']:
                    doc_token.append(token)

            question = chatbot(guess['transcription'])
            

            if 'play' in guess['transcription'].lower():

                youtube(guess['transcription'])
                mytext = 'playing'
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("playing.mp3")
                os.system("playing.mp3")
                break
            
            if any(x in guess['transcription'].lower() for x in ['feel','feeling','I']):
                joke(guess['transcription'])
            
            
            if any(x in guess['transcription'].lower() for x in ['message','whatsapp','text']):
                
                whatsapp(guess['transcription'])
                
            
            if 'time' in guess['transcription'].lower():
                
                mytext = 'the current time is '+ datetime.datetime.now().strftime('%I:%M %p')
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("time.mp3")
                os.system("time.mp3")
                countdown(4)
            
            if any(x in guess['transcription'].lower() for x in ['cook','lunch','recipe','breakfast','meal','dinner','bake']):
                recipe_function()
                
            
            if any(x in guess['transcription'].lower() for x in ['turn','volume']):
                volume_adjust(guess['transcription'])
            
            if 'what' in guess['transcription'].lower():
                describe(guess['transcription'])

                
            for ent in nlp(guess['transcription']).ents:
                if ent.label_ == 'GPE':
                    print('GPE FOUND YES')


                    # 4
                    adj = []

                    for token in nlp(question):

                        if token.text in ['overcast','sunny','clear','rainy','haze','cloudy','broken','scattered','clouds','rain']:
                            adj.append(token)


                    if (len(doc_token) >= 1) & (len(adj) >= 1):
                        similarity_doc_adj = doc_token[-1].similarity(nlp(' '.join([x.text for x in adj])))
                        print()
                        print(f'COSINE SIMILARITY SCORE - {similarity_doc_adj}')
                        print()


                   # 6
                    mytext = chatbot2(guess['transcription'])
                    language = 'en'
                    myobj = gTTS(text=mytext, lang=language, slow=False)
                    myobj.save("response.mp3")
                    os.system("response.mp3")
                    countdown(15)

                elif (ent.label_ == "PERSON") & ('who' in guess['transcription'].lower()):
                    
                    try:

                        mytext = wikipedia.summary(guess['transcription'],2)
                        language = 'en'
                        myobj = gTTS(text=mytext, lang=language, slow=False)
                        myobj.save("response.mp3")
                        os.system("response.mp3")

                    except:
                        print('NO WIKIPEDIA INFORMATION EXTRACTED')

                    finally:
                        pywhatkit.search(ent.text)
                        countdown(round(len(wikipedia.summary(guess['transcription'],2))*0.095))

                elif (ent.label_ == "WORK_OF_ART"):

                    search_keyword=doc2.replace(' ','')
                    html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + search_keyword)
                    video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
                    print("https://www.youtube.com/watch?v=" + video_ids[0])
                    webbrowser.open(f"https://www.youtube.com/watch?v={video_ids[0]}")
                    break




            # 7
            mytext = 'Is there anything else I can help you with?'
            language = 'en'
            myobj = gTTS(text=mytext, lang=language, slow=False)
            myobj.save("next_question.mp3")
            os.system("next_question.mp3")
            countdown(4)
            winsound.Beep(500, 200)

            # 8
            for i in range(100):

                print('Speak!')
                guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                print(guess['transcription'])
                if guess["transcription"]:
                    break
                if not guess["success"]:
                    break

            # loop control
            if 'thank you' in guess['transcription']:

                mytext = 'Thank you Goodbye'
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("Goodbye.mp3")
                os.system("Goodbye.mp3")
                countdown(3)
                loop = False

            else:
                print()
                print('NEXT STATEMENT')

                print('--------------')
                print()
                pass

        except:
            print('the end')
            break

Speak!
hi Helen
Speak!
cook for dinner
cook for dinner
WEATHER BOT
----------
Speak!
let's cook with milk
Speak!
yes please
Speak!
okay
Speak!
okay next
Speak!
okay next
the end
Speak!
None
0: NEXT LOOP
Speak!
None
1: NEXT LOOP
Speak!
None
2: NEXT LOOP
Speak!
None
3: NEXT LOOP
Speak!
None
4: NEXT LOOP
Speak!
None
5: NEXT LOOP
Speak!
None
6: NEXT LOOP
Speak!
None
7: NEXT LOOP
Speak!
None
8: NEXT LOOP
Speak!
None
9: NEXT LOOP
Speak!
None
10: NEXT LOOP
Speak!
None
11: NEXT LOOP
Speak!
None
12: NEXT LOOP
Speak!
None
13: NEXT LOOP
Speak!
hi Helen
Speak!
is it raining in Madrid
is it raining in Madrid
GPE FOUND YES

COSINE SIMILARITY SCORE - 0.2771776299079446

Speak!
that's all thank you
Speak!
None
0: NEXT LOOP
Speak!
None
1: NEXT LOOP
Speak!
None
2: NEXT LOOP
Speak!
None
3: NEXT LOOP
Speak!
None
4: NEXT LOOP
Speak!
None
5: NEXT LOOP
Speak!
None
6: NEXT LOOP
Speak!
None
7: NEXT LOOP
Speak!
None
8: NEXT LOOP
Speak!
None
9: NEXT LOOP
Speak!
None
10: NEXT LOOP
Speak!
None
11: NEXT LOOP
Speak!

KeyboardInterrupt: 

# Looking how the functions work

## Weather Bot V2

**user input - speech to text**

In [7]:
r = sr.Recognizer()
 
with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source)

            print("Please say something")

            audio = r.listen(source)

            print("Recognizing Now .... ")


            # recognize speech using google

            try:
                print("You have said \n" + r.recognize_google(audio))
                print("Audio Recorded Successfully \n ")
                print('-------------------------------')


            except Exception as e:
                print("Error :  " + str(e))


Please say something
Recognizing Now .... 
You have said 
what's the weather in Madrid
Audio Recorded Successfully 
 
-------------------------------


In [41]:
r.recognize_google(audio,show_all=True)

{'alternative': [{'transcript': 'is it raining in Madrid',
   'confidence': 0.89014721}],
 'final': True}

**token classification of user input**

In [42]:
doc = nlp(r.recognize_google(audio))
doc_token = []

print(f'RECORDED STATEMENT - {r.recognize_google(audio)}')
print()
print('TOKENISATION OF RECOREDED STATEMENT')
print()    
for token in doc:

    print(f'{token.text:{10}}{token.pos_:{10}}{spacy.explain(token.pos_):{10}}')
    if token.pos_ in ['ADJ','VERB','NN']:
        doc_token.append(token)
print()
print(f"Identifying GPE in recorded Statement")
displacy.render(doc, style='ent', jupyter=True, options={'distance': 110})
print()
print(f'ADJ,VERB,NN collected - {doc_token}')
print('-----------')

RECORDED STATEMENT - is it raining in Madrid

TOKENISATION OF RECOREDED STATEMENT

is        AUX       auxiliary 
it        PRON      pronoun   
raining   VERB      verb      
in        ADP       adposition
Madrid    PROPN     proper noun

Identifying GPE in recorded Statement



ADJ,VERB,NN collected - [raining]
-----------


**chatbot function to get weather from api**

In [7]:
def chatbot(statement):
    
    #weather = nlp("Current weather in a city")
    statement = nlp(statement)
    print(statement)
    #min_similarity = 0.0

#if weather.similarity(statement) >= min_similarity:
    for ent in statement.ents:
        if ent.label_ == "GPE": # GeoPolitical Entity
            city = ent.text
            city_weather = get_weather(city)

            if city_weather is not None:
                return "In " + city + ", the current weather is: " + city_weather
            else:
                return "Something went wrong."

            break
        else:
            print('NO GPE FOUND')
            pass
    
    print('WEATHER BOT')
    #print(city_weather)
    print('----------')

#chatbot('play middle child')

In [44]:
question = chatbot(r.recognize_google(audio))
question

is it raining in Madrid


'In Madrid, the current weather is: clear sky with temperature 14 celcius......... the minimum temperature for the day will be 13 celcius with maximum temperature 16 celcius'

**token classification of weather statement**

In [45]:
adj = []
print()
print(f'WEATHER STATMENT - {question}')
print()
print('TOKENISATION OF WEATHER STATEMENT')
print()
for token in nlp(question):
    print(f'{token.text:{20}}{token.tag_:{20}}{spacy.explain(token.tag_):{10}}')
    if token.text in ['overcast','sunny','clear','rainy','haze','cloudy','broken','scattered','clouds','rain']:
        adj.append(token)

displacy.render(nlp(question), style='ent', jupyter=True, options={'distance': 110})
print(f'ADJ collected - {adj}')
print('-----------')



WEATHER STATMENT - In Madrid, the current weather is: clear sky with temperature 14 celcius......... the minimum temperature for the day will be 13 celcius with maximum temperature 16 celcius

TOKENISATION OF WEATHER STATEMENT

In                  IN                  conjunction, subordinating or preposition
Madrid              NNP                 noun, proper singular
,                   ,                   punctuation mark, comma
the                 DT                  determiner
current             JJ                  adjective 
weather             NN                  noun, singular or mass
is                  VBZ                 verb, 3rd person singular present
:                   :                   punctuation mark, colon or ellipsis
clear               JJ                  adjective 
sky                 NN                  noun, singular or mass
with                IN                  conjunction, subordinating or preposition
temperature         NN                  noun, singul

ADJ collected - [clear]
-----------


**cosine similarity to determine yes or no, threshold at 0.55**

In [46]:
if (len(doc_token) >= 1) & (len(adj) >= 1):
            similarity_doc_adj = doc_token[-1].similarity(nlp(' '.join([x.text for x in adj])))
            print()
            print(f'COSINE SIMILARITY SCORE - {similarity_doc_adj}')
            print()


COSINE SIMILARITY SCORE - 0.2771776299079446



**redefining chatbot function**

In [7]:
def chatbot2(statement):
            #weather = nlp("Current weather in a city")
            statement = nlp(statement)
            #min_similarity = 0.0

            #if weather.similarity(statement) >= min_similarity:
            for ent in statement.ents:
                if ent.label_ == "GPE": # GeoPolitical Entity
                    city = ent.text
                    break
                else:
                    return "You need to tell me a city to check."
            city_weather = get_weather(city)
            if (city_weather is not None) & (len(doc_token) < 1):
                return "In " + city + ", the current weather is: " + city_weather

            elif (city_weather is not None) & (len(doc_token) >= 1) & (len(adj) >= 1) & (similarity_doc_adj < 0.55):
                return "No.... "+"in " + city + ", the current weather is: " + city_weather

            elif (city_weather is not None) & (len(doc_token) >= 1) & (len(adj) >= 1) & (similarity_doc_adj > 0.55):
                return "Yes.... "+"in " + city + ", the current weather is: " + city_weather
            else:
                return "Something went wrong."
        #else:
                #return "Sorry I don't understand that. Please rephrase your statement."


In [48]:
question = chatbot2(r.recognize_google(audio))
question

'No.... in Madrid, the current weather is: clear sky with temperature 14 celcius......... the minimum temperature for the day will be 13 celcius with maximum temperature 16 celcius'

In [329]:
flight_price(r.recognize_google(audio))

[(8656102463236116519, 0, 1)]
[(8656102463236116519, 0, 1)]


In [21]:
pywhatkit.text_to_handwriting(r.recognize_google(audio))

In [117]:
r = sr.Recognizer()
 
with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source)

            print("Please say something")

            audio = r.listen(source)

            print("Recognizing Now .... ")


            # recognize speech using google

            try:
                print("You have said \n" + r.recognize_google(audio))
                print("Audio Recorded Successfully \n ")
                print('-------------------------------')


            except Exception as e:
                print("Error :  " + str(e))

Please say something
Recognizing Now .... 
You have said 
hi Angel
Audio Recorded Successfully 
 
-------------------------------


## Whatsapp

In [9]:
df_phone_code = pd.read_excel('codes.xlsx',sheet_name='Phone Codes')

In [10]:
phone_codes = dict(zip(df_phone_code['Person'],df_phone_code['Phone Code']))

In [11]:
phone_codes['Eleanor'] = '+34627166596'
phone_codes['Mario'] = '+34681226022'

In [12]:
phone_codes

{'Eleanor': '+34627166596', 'Mario': '+34681226022'}

In [121]:
r = sr.Recognizer()
 
with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source)

            print("Please say something")

            audio = r.listen(source)

            print("Recognizing Now .... ")


            # recognize speech using google

            try:
                print("You have said \n" + r.recognize_google(audio))
                print("Audio Recorded Successfully \n ")
                print('-------------------------------')


            except Exception as e:
                print("Error :  " + str(e))

Please say something
Recognizing Now .... 
You have said 
hi Angel
Audio Recorded Successfully 
 
-------------------------------


In [10]:
def whatsapp(text):
    
    person = []
    for ent in nlp(text).ents:
        if ent.label_ == 'PERSON':
            person.append(ent)
            print(ent,ent.label_)
    
    mytext = "what message would you like to send?"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("whatsapp.mp3")
    os.system("whatsapp.mp3")
    countdown(4)
    winsound.Beep(500, 200)
    
    init_rec = sr.Recognizer()
    for i in range(100):
            
            guess = recognize_speech_from_mic(init_rec, sr.Microphone())
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
    
    hour = datetime.datetime.now().strftime('%H')
    minute = str(int(datetime.datetime.now().strftime('%M')) + 2)
    phone_no = str(phone_codes[str(person[0])])
    message = guess["transcription"]
    time_hour = int(hour)
    time_min = int(minute)

    mytext = "sending"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("whatsapp2.mp3")
    os.system("whatsapp2.mp3")
    countdown(2)
    
    pywhatkit.sendwhatmsg(phone_no=phone_no,message=message,time_hour=time_hour,time_min=time_min)
    
    mytext = "your message has been sent"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)

    
    myobj.save("whatsapp3.mp3")
    os.system("whatsapp3.mp3")
    countdown(4)

In [122]:
person = []

for ent in nlp(r.recognize_google(audio)).ents:
    if ent.label_ == 'PERSON':
        person.append(ent)
        print(ent,ent.label_)

Angel PERSON


In [70]:
r = sr.Recognizer()
 
with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source)

            print("Please say something")

            audio = r.listen(source)

            print("Recognizing Now .... ")


            # recognize speech using google

            try:
                print("You have said \n" + r.recognize_google(audio))
                print("Audio Recorded Successfully \n ")
                print('-------------------------------')


            except Exception as e:
                print("Error :  " + str(e))

Please say something
Recognizing Now .... 
You have said 
hey Mario can get some wine please
Audio Recorded Successfully 
 
-------------------------------


In [71]:
hour = datetime.datetime.now().strftime('%H')

In [72]:
minute = str(int(datetime.datetime.now().strftime('%M')) + 1)

In [73]:
phone_no = str(phone_codes[str(person[0])])

In [74]:
message = r.recognize_google(audio)

In [75]:
time_hour = int(hour)

In [76]:
time_min = int(minute)

In [77]:
pywhatkit.sendwhatmsg(phone_no=phone_no,message=message,time_hour=time_hour,time_min=time_min)

In 6 seconds web.whatsapp.com will open and after 20 seconds message will be delivered


## Time

In [160]:
mytext = 'the current time is '+ datetime.datetime.now().strftime('%I:%M %p')
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("time.mp3")
os.system("time.mp3")
countdown(3)

In [63]:
repeat = True
while repeat:
    
    init_rec = sr.Recognizer()
    for i in range(1000):

        print('Speak!')
        guess = recognize_speech_from_mic(init_rec, sr.Microphone())
        print(guess['transcription'])
        if (guess['transcription'] != None) and ('helen' in guess['transcription'].lower())==True:
            break
        if not guess["success"]:
            break
        print(f"{i}: NEXT LOOP")


    # 1
    mytext = 'Hi.. how can I help you'
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("welcome.mp3")
    os.system("welcome.mp3")
    countdown(4)
    winsound.Beep(500, 200)

    # 2
    for i in range(100):
                print('Speak!')
                guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                print(guess['transcription'])
                if guess["transcription"]:
                    break
                if not guess["success"]:
                    break

    loop = True
    while loop:

        try:


           # 3                
            doc = nlp(guess['transcription'])
            doc_token = []

            for token in doc:
                if token.pos_ in ['ADJ','VERB','NN']:
                    doc_token.append(token)

            question = chatbot(guess['transcription'])
            

            if 'play' in guess['transcription'].lower():

                youtube(guess['transcription'])
                mytext = 'playing'
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("playing.mp3")
                os.system("playing.mp3")
                break
            
            
            if any(x in guess['transcription'].lower() for x in ['message','whatsapp','text']):
                
                whatsapp(guess['transcription'])
                
            
            if 'time' in guess['transcription'].lower():
                
                mytext = 'the current time is '+ datetime.datetime.now().strftime('%I:%M %p')
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("time.mp3")
                os.system("time.mp3")
                countdown(4)
            
            if any(x in guess['transcription'].lower() for x in ['cook','lunch','recipe','breakfast','meal','dinner','bake']):
                recipe_function()                


            for ent in nlp(guess['transcription']).ents:
                if ent.label_ == 'GPE':
                    print('GPE FOUND YES')


                    # 4
                    adj = []

                    for token in nlp(question):

                        if token.text in ['overcast','sunny','clear','rainy','haze','cloudy','broken','scattered','clouds','rain']:
                            adj.append(token)


                    if (len(doc_token) >= 1) & (len(adj) >= 1):
                        similarity_doc_adj = doc_token[-1].similarity(nlp(' '.join([x.text for x in adj])))
                        print()
                        print(f'COSINE SIMILARITY SCORE - {similarity_doc_adj}')
                        print()


                   # 6
                    mytext = chatbot2(guess['transcription'])
                    language = 'en'
                    myobj = gTTS(text=mytext, lang=language, slow=False)
                    myobj.save("response.mp3")
                    os.system("response.mp3")
                    countdown(15)

                elif (ent.label_ == "PERSON") & ('who' in guess['transcription'].lower()):
                    
                    try:

                        mytext = wikipedia.summary(guess['transcription'],2)
                        language = 'en'
                        myobj = gTTS(text=mytext, lang=language, slow=False)
                        myobj.save("response.mp3")
                        os.system("response.mp3")

                    except:
                        print('NO WIKIPEDIA INFORMATION EXTRACTED')

                    finally:
                        pywhatkit.search(ent.text)
                        countdown(round(len(wikipedia.summary(guess['transcription'],2))*0.095))

                elif (ent.label_ == "WORK_OF_ART"):

                    search_keyword=doc2.replace(' ','')
                    html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + search_keyword)
                    video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
                    print("https://www.youtube.com/watch?v=" + video_ids[0])
                    webbrowser.open(f"https://www.youtube.com/watch?v={video_ids[0]}")
                    break




            # 7
            mytext = 'Is there anything else I can help you with?'
            language = 'en'
            myobj = gTTS(text=mytext, lang=language, slow=False)
            myobj.save("next_question.mp3")
            os.system("next_question.mp3")
            countdown(4)
            winsound.Beep(500, 200)

            # 8
            for i in range(100):

                print('Speak!')
                guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                print(guess['transcription'])
                if guess["transcription"]:
                    break
                if not guess["success"]:
                    break

            # loop control
            if 'thank you' in guess['transcription']:

                mytext = 'Thank you Goodbye'
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("Goodbye.mp3")
                os.system("Goodbye.mp3")
                countdown(3)
                loop = False

            else:
                print()
                print('NEXT STATEMENT')

                print('--------------')
                print()
                pass

        except:
            print('the end')
            break

Speak!
None
0: NEXT LOOP
Speak!
None
1: NEXT LOOP
Speak!
None
2: NEXT LOOP
Speak!
hi Helen
Speak!
can I cook for lunch
WEATHER BOT
----------
the end
Speak!
None
0: NEXT LOOP
Speak!
None
1: NEXT LOOP
Speak!
None
2: NEXT LOOP
Speak!
None
3: NEXT LOOP
Speak!
None
4: NEXT LOOP
Speak!
None
5: NEXT LOOP
Speak!
None
6: NEXT LOOP
Speak!
None
7: NEXT LOOP
Speak!
None
8: NEXT LOOP
Speak!
None
9: NEXT LOOP
Speak!
None
10: NEXT LOOP
Speak!
None
11: NEXT LOOP
Speak!
None
12: NEXT LOOP
Speak!
None
13: NEXT LOOP
Speak!
None
14: NEXT LOOP
Speak!
None
15: NEXT LOOP
Speak!
None
16: NEXT LOOP
Speak!


KeyboardInterrupt: 

## Recipe

In [11]:
mytext = 'What accent is this'
language = 'de'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("step1.mp3")
os.system("step1.mp3")
countdown(round(len(mytext)*0.095))

In [52]:
df_recipe = pd.read_excel('recipes.xlsx',sheet_name='chicken')

In [53]:
df_recipe

,Steps,Instructions
0,step 0,Honey Garlic Chicken with brown rice?
1,step 1,"step 1, we will need the following ingredients..."
2,step 2,"step 2, Heat olive oil in a large skillet over..."
3,step 3,"step 3, Lightly season the cubed chicken with ..."
4,step 4,"step 4, Add the chicken to the skillet about 3..."
5,step 5,"step 5, Whisk the honey, soy sauce, garlic and..."
6,step 6,"step 6, Add the sauce to the pan and toss to c..."
7,step 7,"finally, Serve with steamed rice and top with ..."


In [54]:
df_recipe['Instructions'].iloc[0]

'Honey Garlic Chicken with brown rice?'

In [62]:
if any(x in guess['transcription'].lower() for x in ['cook','lunch','recipe','breakfast','meal','dinner','bake'])
#activation code into function

In [35]:
for num in range(len(df_ingrident_exp['text'])):
        
        mytext = df_ingrident_exp['text'].iloc[num]
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save(f"recipe{num}.mp3")
        os.system(f"recipe{num}.mp3")
        countdown(round(len(mytext)*0.095))

In [66]:
def recipe_function():
    
    mytext = "The following ingredients will expire in the coming days"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save(f"recipe_step1.mp3")
    os.system(f"recipe_step1.mp3")
    countdown(round(len(mytext)*0.095))

    #reading the ingridents and when they will expire
    for num in range(len(df_ingrident_exp['text'])):
        
        mytext = df_ingrident_exp['text'].iloc[num]
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save("recipe{num}.mp3")
        os.system("recipe{num}.mp3")
        countdown(round(len(mytext)*0.095))
        
    mytext = "what would you like to cook with?"
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("recipe_step1.mp3")
    os.system("recipe_step1.mp3")
    countdown(round(len(mytext)*0.095))
    
    #ok lets cook with
    init_rec = sr.Recognizer() 
    for i in range(100):
        print('Speak!')
        guess = recognize_speech_from_mic(init_rec, sr.Microphone())
        print(guess['transcription'])
        if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['chicken','coconut']):
            break
    
    for ent in nlp(guess['transcription']).ents:
        df_recipe = pd.read_excel('recipes.xlsx',sheet_name=f"{ent}")
    
    #df_recipe['Instructions'].iloc[0] - recipe name
    mytext = "Would you like to cook " + df_recipe['Instructions'].iloc[0]
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("recipe_step1.mp3")
    os.system("recipe_step1.mp3")
    countdown(round(len(mytext)*0.095))
    
    #yes or no but in our case a definte yes
    init_rec = sr.Recognizer() 
    for i in range(100):
        print('Speak!')
        guess = recognize_speech_from_mic(init_rec, sr.Microphone())
        print(guess['transcription'])
        if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['yes','do','it']):
            break
    
    for num in range(1,len(df_recipe['Instructions'])):
        recepie_loop = True
        while recepie_loop:
            mytext = df_recipe['Instructions'].iloc[num]
            language = 'en'
            myobj = gTTS(text=mytext, lang=language, slow=False)
            myobj.save(f"step{num}.mp3")
            os.system(f"step{num}.mp3")
            countdown(round(len(mytext)*0.095))
            winsound.Beep(500, 200)

            if 'finally' in mytext:
                recepie_loop = False
                break

            else:        
                init_rec = sr.Recognizer() 
                for i in range(100):


                    print('Speak!')
                    guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                    print(guess['transcription'])
                    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['repeat','again']):
                        os.system("repeat.mp3")
                        countdown(1)
                        break

                    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['next','ok','okay']):
                        break

            if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['next','ok','okay']):

                recepie_loop=False
                os.system("next.mp3")
                #countdown(1)
                break

In [65]:
mytext = "The following ingredients will expire in the coming days"
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("recipe_step1.mp3")
os.system("recipe_step1.mp3")
countdown(round(len(mytext)*0.095))

#reading the ingridents and when they will expire
for num in range(len(df_ingrident_exp['text'])):

    mytext = df_ingrident_exp['text'].iloc[num]
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save(f"recipe{num}.mp3")
    os.system(f"recipe{num}.mp3")
    countdown(round(len(mytext)*0.095))

mytext = "what would you like to cook with?"
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("recipe_step1.mp3")
os.system("recipe_step1.mp3")
countdown(round(len(mytext)*0.095))

#ok lets cook with
init_rec = sr.Recognizer() 
for i in range(100):
    print('Speak!')
    guess = recognize_speech_from_mic(init_rec, sr.Microphone())
    print(guess['transcription'])
    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['chicken','coconut']):
        break

for ent in nlp(guess['transcription']).ents:
    df_recipe = pd.read_excel('recipes.xlsx',sheet_name=f"{ent}")

#df_recipe['Instructions'].iloc[0] - recipe name
mytext = "Would you like to cook " + df_recipe['Instructions'].iloc[0]
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("recipe_step1.mp3")
os.system("recipe_step1.mp3")
countdown(round(len(mytext)*0.095))

#yes or no but in our case a definte yes
init_rec = sr.Recognizer() 
for i in range(100):
    print('Speak!')
    guess = recognize_speech_from_mic(init_rec, sr.Microphone())
    print(guess['transcription'])
    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['yes','do','it']):
        break

for num in range(1,len(df_recipe['Instructions'])):
    recepie_loop = True
    while recepie_loop:
        mytext = df_recipe['Instructions'].iloc[num]
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save(f"step{num}.mp3")
        os.system(f"step{num}.mp3")
        countdown(round(len(mytext)*0.095))
        winsound.Beep(500, 200)

        if 'finally' in mytext:
            recepie_loop = False
            break

        else:        
            init_rec = sr.Recognizer() 
            for i in range(100):


                print('Speak!')
                guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                print(guess['transcription'])
                if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['repeat','again']):
                    os.system("repeat.mp3")
                    countdown(1)
                    break

                if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['next','ok','okay']):
                    break

        if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['next','ok','okay']):

            recepie_loop=False
            os.system("next.mp3")
            #countdown(1)
            break

Speak!
let's cook with chicken
Speak!
None
Speak!
None
Speak!
yes Nickelback
Speak!
can you repeat that
Speak!
okay next
Speak!
okay next
Speak!
okay next
Speak!
okay next
Speak!
okay next
Speak!
okay next


In [19]:
for ent in nlp('chicken,coconut').ents:
    print(ent,ent.label_)

chicken PRODUCT
coconut PRODUCT


In [26]:
#then yes and no but in our case yes try to look at alternative

In [161]:
mytext = 'then'
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("next.mp3")
os.system("next.mp3")

0

In [162]:
mytext = 'sure'
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("repeat.mp3")
os.system("repeat.mp3")

0

In [98]:
for num in range(1,len(df_recipe['Instructions'])):
    
    recepie_loop = True
    
    while recepie_loop:
        
        mytext = df_recipe['Instructions'].iloc[num]
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save(f"step{num}.mp3")
        os.system(f"step{num}.mp3")
        countdown(round(len(mytext)*0.095))
        winsound.Beep(500, 200)

        if 'finally' in mytext:
            recepie_loop = False
            break

        else:        
            init_rec = sr.Recognizer() 
            for i in range(100):


                print('Speak!')
                guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                print(guess['transcription'])
                if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['repeat','again']):
                    os.system("repeat.mp3")
                    countdown(1)
                    break
                
                if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['next','ok','okay']):
                    break

        if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['next','ok','okay']):

            recepie_loop=False
            os.system("next.mp3")
            #countdown(1)
            break

NameError: name 'df_recipe' is not defined

In [51]:
now = datetime.datetime.now()
c_t = now.strftime('%H:%M:%S')

In [52]:
c_t

'02:01:24'

## Flight

In [163]:
import pandas as pd
df_city_codes = pd.read_excel('codes.xlsx',sheet_name='City Codes')
df_month_codes = pd.read_excel('codes.xlsx',sheet_name='Month Codes')

In [164]:
city_code = dict(zip(df_city_codes['City'],df_city_codes['Code']))
month_code = dict(zip(df_month_codes['Month'],df_month_codes['Code']))

In [165]:
month_code['Janurary'] = '01'
month_code['Feburary'] = '02'
month_code['March'] = '03'
month_code['April'] = '04'
month_code['May'] = '05'
month_code['June'] = '06'
month_code['July'] = '07'
month_code['August'] = '08'
month_code['September'] = '09'

In [11]:
r = sr.Recognizer()
 
with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source)

            print("Please say something")

            audio = r.listen(source)

            print("Recognizing Now .... ")


            # recognize speech using google

            try:
                print("You have said \n" + r.recognize_google(audio))
                print("Audio Recorded Successfully \n ")
                print('-------------------------------')


            except Exception as e:
                print("Error :  " + str(e))

Please say something
Recognizing Now .... 
You have said 
can you please send Jose Mario a message
Audio Recorded Successfully 
 
-------------------------------


In [12]:
displacy.render(nlp(r.recognize_google(audio)), style='ent', jupyter=True, options={'distance': 110})

In [13]:
input_text = nlp(r.recognize_google(audio))

In [14]:
for ent in input_text.ents:
    print(ent)

Jose Mario


In [275]:
city = []
dates = []

for ent in input_text.ents:
    
    if ent.label_ == "GPE":
        city.append(ent)
        #print(city)
    elif ent.label_ == "DATE":
        dates.append(ent)

In [270]:
city_code[str(city[0])]

'MAD'

In [271]:
str(dates).replace('th','').replace('nd','').replace('rd','')

'[May 24 till, May 25]'

In [195]:
[int(s) for s in str(dates[0]).replace('th','').replace('nd','').replace('rd','').split() if s.isdigit()]

24

In [172]:
str(dates[0][0]).replace('th','').replace('nd','').replace('rd','')

'24'

In [173]:
month_code[str(dates[0][1])]+'-'+str(dates[0][0]).replace('th','').replace('nd','').replace('rd','')

'04-24'

In [174]:
month_code[str(dates[1][1])]+'-'+str(dates[1][0]).replace('th','').replace('nd','').replace('rd','')

'04-24'

In [175]:
def flight_price(x):
    
    input_text = nlp(x)
    
    city = []
    dates = []
    
    for ent in input_text.ents:
        if ent.label_ == "GPE":
            city.append(ent)
        elif ent.label_ == "DATE":
            dates.append(ent)
    
    city_from = city_code[str(city[0])]
    city_to = city_code[str(city[1])]
    
    dates = str(dates).replace('th','').replace('nd','').replace('rd','')
    
    date_from = month_code[str(dates[0][1])]+'-'+str(dates[0][0]).replace('th','').replace('nd','').replace('rd','')
    date_to = month_code[str(dates[1][1])]+'-'+str(dates[1][0]).replace('th','').replace('nd','').replace('rd','')
    
    url = f"https://flights.booking.com/flights/{city_from}-{city_to}/?type=ROUNDTRIP&adults=1&cabinClass=ECONOMY&children=&from={city_from}&to={city_to}&depart=2021-{date_from}&return=2021-{date_to}&sort=BEST&aid=304142&label=gen173nr-1DCAEoggI46AdIM1gEaEaIAQGYATG4ARfIAQzYAQPoAQH4AQKIAgGoAgO4ArW2_YMGwAIB0gIkMTExOTA1ZmEtMGE1Zi00OGI4LTk0MjYtYWUwZjIzOGM2YjE22AIE4AIB"
    
    webbrowser.open(url)
            
    

In [328]:
def flight_price(x):
    
    input_text = nlp(x)
    
    city = []
    dates = []
    
    for ent in input_text.ents:
        if ent.label_ == "GPE":
            city.append(ent)
        elif ent.label_ == "DATE":
            dates.append(ent)
    
    city_from = city_code[str(city[0])]
    city_to = city_code[str(city[1])]
    
    from spacy.matcher import Matcher
    matcher = Matcher(nlp.vocab)
    pattern1 = [{'LOWER':'janurary'}]
    pattern2 = [{'LOWER':'feburary'}]
    pattern3 = [{'LOWER':'march'}]
    pattern4 = [{'LOWER':'april'}]
    pattern5 = [{'LOWER':'may'}]
    pattern6 = [{'LOWER':'june'}]
    pattern7 = [{'LOWER':'july'}]
    pattern8 = [{'LOWER':'august'}]
    pattern9 = [{'LOWER':'september'}]
    pattern10 = [{'LOWER':'october'}]
    pattern11 = [{'LOWER':'november'}]
    pattern12 = [{'LOWER':'december'}]
    matcher.add('flight',patterns=[pattern1,pattern2,pattern3,pattern4,pattern5,pattern6,pattern7,pattern8,pattern9,pattern10,pattern11,pattern12],on_match=None)
    found_matches1 = matcher(dates[0])
    found_matches2 = matcher(dates[1])
    #print(found_matches1)
    #print(found_matches2)
    
    #dates = str(dates).replace('th','').replace('nd','').replace('rd','')
    
    date_from = month_code[str(dates[0][found_matches1[0][1]])]+'-'+str([int(s) for s in str(dates[0]).replace('th','').replace('nd','').replace('rd','').split() if s.isdigit()][0])
    date_to = month_code[str(dates[1][found_matches2[0][1]])]+'-'+str([int(s) for s in str(dates[1]).replace('th','').replace('nd','').replace('rd','').split() if s.isdigit()][0])
    
    url = f"https://flights.booking.com/flights/{city_from}-{city_to}/?type=ROUNDTRIP&adults=1&cabinClass=ECONOMY&children=&from={city_from}&to={city_to}&depart=2021-{date_from}&return=2021-{date_to}&sort=BEST&aid=304142&label=gen173nr-1DCAEoggI46AdIM1gEaEaIAQGYATG4ARfIAQzYAQPoAQH4AQKIAgGoAgO4ArW2_YMGwAIB0gIkMTExOTA1ZmEtMGE1Zi00OGI4LTk0MjYtYWUwZjIzOGM2YjE22AIE4AIB"
    
    webbrowser.open(url)
            
    

## Jokes

In [82]:
from random import randint

In [83]:
#activation code
#if (x in guess['transcription'].lower() for x in ['feel','feeling'])

In [86]:
text = 'I feel sad'

In [116]:
mytext = "Well we can't have you feeling that way the whole day, do you want to hear a joke?"
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("joke.mp3")
os.system("joke.mp3")
countdown(7)
winsound.Beep(500, 200)

In [117]:
def joke(text):
    
    if sia.polarity_scores(text)['compound'] > 0.1:

        mytext = 'Call me crazy but I love to see other people happy, It makes me happy'
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save("happy.mp3")
        os.system("happy.mp3")
        countdown(round(len(mytext)*0.095))

    if sia.polarity_scores(text)['compound'] < 0.1:

        mytext = "Well we can't have you feeling that way the whole day, do you want to hear a joke?"
        language = 'en'
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save("joke.mp3")
        os.system("joke.mp3")
        countdown(7)
        winsound.Beep(500, 200)

        init_rec = sr.Recognizer() 
        for i in range(100):
            print('Speak!')
            guess = recognize_speech_from_mic(init_rec, sr.Microphone())
            print(guess['transcription'])
            if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['yes','no','ok','sure','tell']):
                break
            if not guess["success"]:
                break

        if any(x in guess['transcription'].lower() for x in ['yes','please','sure','tell']):

            joke = True
            while joke:


                df_joke = pd.read_excel('jokes v1.xlsx')
                mytext = df_joke['Jokes'][randint(0,len(df_joke['Jokes']))]
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("joke_text.mp3")
                os.system("joke_text.mp3")
                countdown(round(len(mytext)*0.095))

                mytext = "lol,would you like to hear another one?"
                language = 'en'
                myobj = gTTS(text=mytext, lang=language, slow=False)
                myobj.save("joke_again.mp3")
                os.system("joke_again.mp3")
                countdown(round(len(mytext)*0.095))
                winsound.Beep(500, 200)

                init_rec = sr.Recognizer() 
                for i in range(100):

                    print('Speak!')
                    guess = recognize_speech_from_mic(init_rec, sr.Microphone())
                    print(guess['transcription'])

                    if guess["transcription"] and any(x in guess['transcription'].lower() for x in ['yes','ok','sure','tell']):
                        break
                    if guess["transcription"] and ('no' in guess['transcription'].lower()):
                        break

                if guess["transcription"] and ('no' in guess['transcription'].lower()):
                        joke = False


Speak!
no thank you


In [92]:
df_joke['Jokes'][randint(0,len(df_joke['Jokes']))]

'How does Michael Jackson pick his nose?\xa0From a catalogue.'

In [57]:
df_joke = pd.read_excel('jokes v1.xlsx')

In [62]:
df_joke['Jokes'][0]

'Did you hear the pasta makers in Italy revolted?It was a ravioli-tion.'

In [54]:
init_rec = sr.Recognizer() 
for i in range(100):
    print('Speak!')
    guess = recognize_speech_from_mic(init_rec, sr.Microphone())
    print(guess['transcription'])
    if any(x in guess['transcription'].lower() for x in ['yes','no','ok']):
        break


Speak!
okay
Speak!
yes


In [82]:
mytext = 'How does Michael Jackson pick his nose?\xa0From a catalogue.'
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("joke.mp3")
os.system("joke.mp3")
countdown(round(len(mytext)*0.095))
    

In [105]:
doc = nlp('who is David Beckham')

In [107]:
displacy.render(doc, style='ent', jupyter=True, options={'distance': 110})

In [109]:
#recipe_function()

In [8]:
pywhatkit.info('google',lines=1)

PageError: Page id "google\" does not match any pages. Try another id!

**Positive Sentiment**

In [52]:
sia.polarity_scores('I feel really happy today')

{'neg': 0.0, 'neu': 0.429, 'pos': 0.571, 'compound': 0.6115}

**Negative Sentiment**

In [50]:
sia.polarity_scores('I feel really sad today')

{'neg': 0.531, 'neu': 0.469, 'pos': 0.0, 'compound': -0.5256}

In [36]:
mytext = "The following ingredients will expire in the coming days" + str(list(df_ingredient_exp['text']))
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=False)
myobj.save("joke.mp3")
os.system("joke.mp3")
countdown(round(len(mytext)*0.095))

In [47]:
list(df_ingrident['Ingredient'])

['salt',
 'sugar',
 'water',
 'oil and fat',
 'flavouring',
 'dairy',
 'vegetable oil and fat',
 'cereal',
 'vegetable',
 'fruit',
 'flour',
 'wheat',
 'root vegetable',
 'vegetable oil',
 'glucose',
 'starch',
 'cereal flour',
 'milk',
 'natural flavouring',
 'emulsifier',
 'spice',
 'wheat flour',
 'e330',
 'herb',
 'seed',
 'sunflower oil',
 'colour',
 'preservative',
 'cocoa',
 'acid',
 'e322',
 'plant',
 'onion',
 'antioxidant',
 'egg',
 'dextrose',
 'thickener',
 'milk powder',
 'e322i',
 'yeast',
 'rapeseed oil',
 'garlic',
 'pepper',
 'glucose syrup',
 'stabiliser',
 'citrus fruit',
 'animal',
 'protein',
 'tomato',
 'nut',
 'fruit juice',
 'corn starch',
 'palm oil and fat',
 'cocoa butter',
 'acidity regulator',
 'cream',
 'soya lecithin',
 'cocoa paste',
 'raising agent',
 'vinegar',
 'meat',
 'e500',
 'skimmed milk powder',
 'rice',
 'e300',
 'modified starch',
 'colza oil',
 'ferment',
 'pork',
 'fat',
 'malt',
 'lemon',
 'tree nut',
 'sea salt',
 'e415',
 'cocoa powder',


In [23]:
df_joke = pd.read_excel('jokes v1.xlsx')

In [28]:
df_joke['Jokes']

0     Did you hear the pasta makers in Italy revolte...
1     What are Mexicans favorite sport ? Cross country.
2     Why do the French eat snails? They don’t like ...
3     How do you get a pig moving in Spain?You say J...
4     Someone stole my microsoft office and they're ...
5     What did Jay-Z call his wife before they got m...
6     How do you find Will Smith in a snowstorm? Loo...
7     How does Michael Jackson pick his nose? From a...
8     What do you call it when Batman skips church? ...
9     Why did Mickey go into outer space?To find Pluto!
10    When Chuck Norris does a push-up, he isn't lif...
11    You should always knock on the fridge door bef...
12    I broke my finger last week. On the other hand...
Name: Jokes, dtype: object

In [30]:
df_joke = pd.read_excel('jokes v1.xlsx')
for num in range(len(df_joke['Jokes'])):
    
    mytext = df_joke['Jokes'].iloc[num]
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save(f"joke{num}.mp3")
    os.system(f"joke{num}.mp3")
    countdown(round(len(mytext)*0.095))